## Prompt flow RAG example
---

Amazon Bedrock Flows accelerates the creation, testing, and deployment of user-defined workflows for generative AI applications through an intuitive visual builder. Using Bedrock prompt flows, users can drag, drop and link Prompts, existing Agents, Knowledge Bases, Guardrails and other AWS services. This enables generative AI teams to build a business logic via workflow creations. 

In this example, we will be building a simple RAG application. We will follow the following steps:

1. We will be creating a RAG prompt. This prompt will be stored in Amazon Bedrock within the prompt management feature functionality. Prompt management on Bedrock simplifies the creation, evaluation, versioning, and sharing of prompts to help developers and prompt engineers get the best responses from foundation models (FMs) for their use cases. 

1. Next, we will create a knowledge base, which will store sample information about AWS services. 

1. We will create a prompt flow with the user provided prompt, retrieval of chunks from the knowledge base via a lambda function, followed by answering the user defined question. We will also introduce Guardrails and update our flow at the end.

1. Lastly, we want to be able to access the prompt flow from an external application via an API or export it via a CloudFormation template. We will be doing so at the end. This helps developers to seamlessly access their RAG applications through a simple invocation.


[Amazon Bedrock Prompt Management](https://docs.aws.amazon.com/bedrock/latest/userguide/prompt-management.html) streamlines the creation, evaluation, deployment, and sharing of prompts in the Amazon Bedrock console and via APIs in the SDK. This feature helps developers and business users obtain the best responses from foundation models for their specific use cases.

[Amazon Bedrock Prompt Flows](https://docs.aws.amazon.com/bedrock/latest/userguide/flows.html) allows you to easily link multiple foundation models (FMs), prompts, and other AWS services, reducing development time and effort. It introduces a visual builder in the Amazon Bedrock console and a new set of APIs in the SDK, that simplifies the creation of complex generative AI workflows.

Let's start by making sure we have the lastest version of the Amazon Bedrock SDK, importing the libraries, and setting-up the client.

In [ ]:
# Only run this the first time...
!pip3 install boto3 botocore matplotlib -qU

In [ ]:
import os
import json
import boto3
import logging
import requests
from datetime import datetime

In [ ]:
# Set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# Define constants

# This is the model that will be used for standard text generation for our
# RAG application use case
TEXT_GEN_MODEL: str = "amazon.nova-lite-v1:0"
# S3 bucket that contains information on the PDF file that is stored within the Knowledge base
S3_INPUT_BUCKET: str = "genai-appdev-frameworks"
# This is the link of the PDF file that is stored within the Knowledge base
URL: str = 'https://d0.awsstatic.com/whitepapers/aws-overview.pdf'
# Local directory where the data is saved
DATA_DIR: str = "data"
# PDF file name to be saved and put in S3
PDF_FILE_NAME: str = "aws-overview.pdf"
# This is the name of the Knowledge base that will be created
KB_NAME: str = "aws-overview-kb"
KB_DESC: str = """Use this KB to retrieve information about AWS services that the user is asking about."""

# This is the lambda function that will be used to fetch the top 5 chunks from the knowledge base
KB_LAMBDA_FUNCTION_NAME: str = 'demo-lambda-function'
KB_LAMBDA_FUNCTION: str = 'kb-lambda-function.py'

#### Save the file locally, put it in S3, and then sync it with the Knowledge base

In [ ]:
# Download the file
try:
    if not os.path.exists(DATA_DIR):
        os.makedirs(DATA_DIR)
    response = requests.get(URL)
    response.raise_for_status()
    
    # Save the file
    file_path = os.path.join(DATA_DIR, PDF_FILE_NAME)
    with open(file_path, 'wb') as f:
        f.write(response.content)
    logger.info(f"File successfully downloaded to {file_path}")
except requests.exceptions.RequestException as e:
    logger.info(f"Error downloading file: {e}")

In [ ]:
s3_client = boto3.client('s3')
# Next, put the data contents in the input s3 bucket. We will use this s3 bucket content to sync with the knowledge base after 
# creating it
try:
    file_path = os.path.join(DATA_DIR, PDF_FILE_NAME)
    s3_client.upload_file(
        file_path,           
        S3_INPUT_BUCKET,     
        PDF_FILE_NAME 
    )
    logger.info(f"Successfully uploaded {PDF_FILE_NAME} to {S3_INPUT_BUCKET}")
except Exception as e:
    logger.info(f"Error uploading to S3: {e}")

In [ ]:
# fetch the current AWS region
region = boto3.client('sts').meta.region_name # change to another region if you do not want
# the region to be dynamically fetched
logger.info(f"Current AWS region: {region}")

In [ ]:
bedrock_agent = boto3.client(service_name = "bedrock-agent", region_name = region)

### RAG Application Prompt

Let's create our sample RAG application prompt by leveraging on Prompt Management for Amazon Bedrock. Here, you can adjust the sample prompt template.

In [ ]:
response = bedrock_agent.create_prompt(
    name = f"aws-expert-assistant",
    description = "Prompt template for AWS expert assistant to analyze content and answer user questions",
    variants = [
        {
            "inferenceConfiguration": {
            "text": {
                "maxTokens": 2000,
                "temperature": 0,
            }
            },
            "modelId": TEXT_GEN_MODEL,
            "name": "variantOne",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {
                            "name": "question"
                        },
                        {
                            "name": "context"
                        }
                    ],
                    "text": """
<system>You are an expert AWS assistant with deep knowledge of AWS services, architectures, and best practices. Your role is to carefully analyze provided content and answer user questions with accurate, detailed, and technically precise information.</system>

Refer to the user question below in the user_question xml tags:
<user_question>
{{question}}
</user_question>

Here is the content to analyze to answer the user question in the content xml tags:
<content>xw
{{context}}
</content>

<assistant>I'll help you by analyzing the content provided and answering your specific question about AWS. Let me break this down systematically:

1. First, I'll carefully examine your question and the provided content
2. Then, I'll provide a detailed, technically accurate answer
3. I'll ensure to include relevant AWS-specific details and best practices
4. If any information is unclear or missing, I'll note that in my response

Here's my answer:</assistant>
                    """
                }
            },
            "templateType": "TEXT"
        }
    ],
    defaultVariant = "variantOne"
)
logger.info(json.dumps(response, indent=2, default=str))
promptEvalId = response["id"]
promptEvalArn = response["arn"]
promptEvalName = response["name"]
logger.info(f"Prompt ID: {promptEvalId}\nPrompt ARN: {promptEvalArn}\nPrompt Name: {promptEvalName}")

Now that we have a draft prompt, we can create a version from it.

In [ ]:
response = bedrock_agent.create_prompt_version(
    promptIdentifier = promptEvalId
)
print(json.dumps(response, indent=2, default=str))

### Create a Knowledge Base
---

In this portion of the notebook, we will create a knowledge base and store information about AWS services.

In [ ]:
import sys
sys.path.insert(0, ".")
from utils import *
from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
from utils.lambda_utils import *

# Initialize the KB class
kb = KnowledgeBasesForAmazonBedrock()

In [ ]:
%%time
try:
    kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
        KB_NAME,
        KB_DESC,
        S3_INPUT_BUCKET, 
    )
    logger.info(f"Knowledge Base ID: {kb_id}")
    logger.info(f"Data Source ID: {ds_id}")
except Exception as e:
    logger.error(f"An error occurred while creating the KB: {e}")

### Synchronize the Knowledge Base with Data in `S3`
---

Now we will sync the data from the S3 bucket into the AWS expert knowledge base

In [ ]:
# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

### Wrap the knowledge base in a lambda function
---

Next, we will wrap the function above in a lambda function and then invoke the lambda function to get the responses

In [ ]:
# Deploy the Lambda. This lambda will return the top 5 chunks 
# from the knowledge base that contains information on the data that
# is inputted
function_arn = create_kb_lambda(
    lambda_function_name=KB_LAMBDA_FUNCTION_NAME,
    source_code_file=os.path.join('utils', KB_LAMBDA_FUNCTION),
    region=region,
    kb_id=kb_id
)

logger.info(f"Lambda function ARN: {function_arn}")

### RAG Application Flow
---

Now, we have created a RAG Application prompt, a knowledge base that contains information about AWS services, and have wrapped the knowledge base in a lambda function so that when the user asks a question, the top 5 chunks from the knowledge base are retrieved. Once that is done, the chunks and the user question are fed into a customized prompt and the answer is given back to the user.


We'll need an AWS IAM role for creating the Prompt Flow in Amazon Bedrock. If you already have a role with your permissions you can directly replace the ```flowRole``` variable with your role's ARN.

For simplicity in this example we'll create a new role and attach the ```AmazonBedrockFullAccess``` policy to it. In general, it's recommended that you further limit the policies with conditions.

You can check further details in the [How Prompt Flows for Amazon Bedrock works](https://docs.aws.amazon.com/bedrock/latest/userguide/flows-how-it-works.html) documentation.

In [ ]:
import boto3
import json

iam = boto3.client('iam')

# Create the role if it doesn't exist
try:
    response = iam.create_role(
        RoleName='MyBedrockFlowsRole',
        AssumeRolePolicyDocument=json.dumps({
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {
                        "Service": [
                            "bedrock.amazonaws.com",
                            "lambda.amazonaws.com"
                        ]
                    },
                    "Action": "sts:AssumeRole"
                }
            ]
        })
    )
    flowRole = response['Role']['Arn']
except iam.exceptions.EntityAlreadyExistsException:
    flowRole = f"arn:aws:iam::{boto3.client('sts').get_caller_identity()['Account']}:role/MyBedrockFlowsRole"

# Attach necessary policies
policies_to_attach = [
    'arn:aws:iam::aws:policy/AmazonBedrockFullAccess',
    'arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
]

for policy in policies_to_attach:
    try:
        iam.attach_role_policy(
            RoleName='MyBedrockFlowsRole',
            PolicyArn=policy
        )
    except iam.exceptions.NoSuchEntityException:
        print(f"Role doesn't exist for policy: {policy}")
    except iam.exceptions.LimitExceededException:
        print(f"Policy already attached: {policy}")

# Create inline policy for Lambda invocation
lambda_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "lambda:InvokeFunction"
            ],
            "Resource": [
                function_arn
            ]
        }
    ]
}

try:
    iam.put_role_policy(
        RoleName='MyBedrockFlowsRole',
        PolicyName='LambdaInvokePolicy',
        PolicyDocument=json.dumps(lambda_policy)
    )
except Exception as e:
    print(f"Error attaching Lambda invoke policy: {e}")

logger.info(f'Using flowRole: {flowRole}')


In [ ]:
response = bedrock_agent.create_flow(
    name=f"RAGFlow-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    description="RAG Application Flow that uses KB and custom prompt to answer questions",
    executionRoleArn=flowRole,
    definition={
        "nodes": [
            # Start Node - receives the user question
            # This is the entry point. Here, the user asks a question, and the output is the 
            # document string that is to be used by the other nodes.
            {
                "name": "Start",
                "type": "Input",
                "configuration": {
                    "input": {}
                },
                "outputs": [
                    {
                        "name": "document",
                        "type": "String"
                    }
                ]
            },
            
            # Lambda Node to get KB chunks - this is the second node. It calls a lambda function to retrieve the 
            # relevant top 5 chunks from the knowledge base
            {
                "name": "GetKBChunks",
                "type": "LambdaFunction",
                "configuration": {
                    "lambdaFunction": {
                        "lambdaArn": function_arn
                    }
                },
                "inputs": [
                    {
                        "expression": "$.data",
                        "name": "input",
                        "type": "String"
                    }
                ],
                "outputs": [
                    {
                        "name": "functionResponse",
                        "type": "Object"
                    }
                ]
            },
            
            # This is the third node. It uses the prompt template stored in the prompt management store
            # and injects it with the user question and chunks retrieved from the lambda function.
            # Next, it generates the answer to the user question.
            {
                "name": "GenerateResponse",
                "type": "Prompt",
                "configuration": {
                    "prompt": {
                        "sourceConfiguration": {
                            "resource": {
                                "promptArn": promptEvalArn
                            }
                        }
                    }
                },
                "inputs": [
                    {
                        "expression": "$.data",  
                        "name": "input",
                        "type": "String"
                    },
                    {
                        "expression": "$.data.GetKBChunks.functionResponse.document",
                        "name": "context",
                        "type": "String"
                    }
                ],
                "outputs": [
                    {
                        "name": "modelCompletion",
                        "type": "String"
                    }
                ]
            },
            
            # End Node
            {
                "name": "End",
                "type": "Output",
                "configuration": {
                    "output": {}
                },
                "inputs": [
                    {
                        "expression": "$.data.GenerateResponse.modelCompletion",
                        "name": "document",
                        "type": "String"
                    }
                ]
            }
        ],
        "connections": [
            {
                "name": "StartToLambda",
                "source": "Start",
                "target": "GetKBChunks",
                "type": "Data",
                "configuration": {
                    "data": {
                        "sourceOutput": "document",
                        "targetInput": "input"
                    }
                }
            },
            {
                "name": "StartToPrompt",
                "source": "Start",
                "target": "GenerateResponse",
                "type": "Data",
                "configuration": {
                    "data": {
                        "sourceOutput": "document",
                        "targetInput": "input"
                    }
                }
            },
            {
                "name": "LambdaToPrompt",
                "source": "GetKBChunks",
                "target": "GenerateResponse",
                "type": "Data",
                "configuration": {
                    "data": {
                        "sourceOutput": "functionResponse",
                        "targetInput": "context"
                    }
                }
            },
            {
                "name": "PromptToEnd",
                "source": "GenerateResponse",
                "target": "End",
                "type": "Data",
                "configuration": {
                    "data": {
                        "sourceOutput": "modelCompletion",
                        "targetInput": "document"
                    }
                }
            }
        ]
    }
)

flowId = response["id"]
flowArn = response["arn"]
flowName = response["name"]
logger.info(json.dumps(response, indent=2, default=str))
logger.info(f"Flow ID: {flowId}\nFlow ARN: {flowArn}\nFlow Name: {flowName}")

In [ ]:
response = bedrock_agent.create_flow(
    name=f"RAGFlow-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    description="RAG Application Flow that uses KB and custom prompt to answer questions",
    executionRoleArn=flowRole,
    definition={
        "nodes": [
            {
                "name": "Start",
                "type": "Input",
                "configuration": {
                    "input": {}
                },
                "outputs": [
                    {
                        "name": "document",
                        "type": "String"
                    }
                ]
            },
            {
                "name": "GetKBChunks",
                "type": "LambdaFunction",
                "configuration": {
                    "lambdaFunction": {
                        "lambdaArn": function_arn
                    }
                },
                "inputs": [
                    {
                        "expression": "$.data",
                        "name": "input",
                        "type": "String"
                    }
                ],
                "outputs": [
                    {
                        "name": "functionResponse",
                        "type": "String"
                    }
                ]
            },
            {
                "name": "GenerateResponse",
                "type": "Prompt",
                "configuration": {
                    "prompt": {
                        "sourceConfiguration": {
                            "resource": {
                                "promptArn": promptEvalArn
                            }
                        }
                    }
                },
                "inputs": [
                    {
                        "expression": "$.data",
                        "name": "question",
                        "type": "String"
                    },
                    {
                        "expression": "$.data",
                        "name": "context",
                        "type": "String"
                    }
                ],
                "outputs": [
                    {
                        "name": "modelCompletion",
                        "type": "String"
                    }
                ]
            },
            {
                "name": "End",
                "type": "Output",
                "configuration": {
                    "output": {}
                },
                "inputs": [
                    {
                        "expression": "$.data",
                        "name": "document",
                        "type": "String"
                    }
                ]
            }
        ],
        "connections": [
            {
                "name": "StartToLambda",
                "source": "Start",
                "target": "GetKBChunks",
                "type": "Data",
                "configuration": {
                    "data": {
                        "sourceOutput": "document",
                        "targetInput": "input"
                    }
                }
            },
            {
                "name": "StartToPrompt",
                "source": "Start",
                "target": "GenerateResponse",
                "type": "Data",
                "configuration": {
                    "data": {
                        "sourceOutput": "document",
                        "targetInput": "question" 
                    }
                }
            },
            {
                "name": "LambdaToPrompt",
                "source": "GetKBChunks",
                "target": "GenerateResponse",
                "type": "Data",
                "configuration": {
                    "data": {
                        "sourceOutput": "functionResponse",
                        "targetInput": "context"
                    }
                }
            },
            {
                "name": "PromptToEnd",
                "source": "GenerateResponse",
                "target": "End",
                "type": "Data",
                "configuration": {
                    "data": {
                        "sourceOutput": "modelCompletion",
                        "targetInput": "document"
                    }
                }
            }
        ]
    }
)


flowId = response["id"]
flowArn = response["arn"]
flowName = response["name"]
logger.info(json.dumps(response, indent=2, default=str))
logger.info(f"Flow ID: {flowId}\nFlow ARN: {flowArn}\nFlow Name: {flowName}")

This is what your Bedrock prompt flow should look like on the console:

![bedrock-prompt-flow](prompt_flow_1.png)

In [ ]:
# prepare teh flow, so that it builds and validates our flow creation
response = bedrock_agent.prepare_flow(
    flowIdentifier = flowId
)
logger.info(json.dumps(response, indent=2, default=str))

In [ ]:
response = bedrock_agent.get_flow(
    flowIdentifier = flowId
)
logger.info(json.dumps(response, indent=2, default=str))

In [ ]:
response = bedrock_agent.create_flow_version(
    flowIdentifier = flowId
)
logger.info(json.dumps(response, indent=2, default=str))

In [ ]:
# We can also create flow alises, so that we can point our application front-ends and any other integrations to these. 
# This allows creating new versions without impacting our service.
response = bedrock_agent.create_flow_alias(
    flowIdentifier = flowId,
    name = flowName,
    description = "Alias for my test flow in the customer service use case",
    routingConfiguration = [
        {
            "flowVersion": "1"
        }
    ]
)
logger.info(json.dumps(response, indent=2, default=str))
flowAliasId = response['id']

### Invoke a Flow
---

Now that we have learned how to create and manage flows, we can test these with invocations.

**Note**: You can invoke flows from any application front-end or your own systems as required. It effectively exposes all the logic of your flow through an Agent Endpoint API.

In [ ]:
bedrock_agent_runtime = boto3.client(service_name = 'bedrock-agent-runtime', region_name = 'us-east-1')

In [173]:
response = bedrock_agent_runtime.invoke_flow(
    flowIdentifier = flowId,
    flowAliasIdentifier = flowAliasId,
    inputs = [
        { 
            "content": { 
                "document": "Hi, What is Amazon SageMaker?"
            },
            "nodeName": "Start",
            "nodeOutputName": "document"
        }
    ]
)

# Process the event stream
for event in response["responseStream"]:
    if "flowOutputEvent" in event:
        # Extract just the answer content
        answer = event["flowOutputEvent"]["content"]["document"]
        # Remove the <answer> tags if present
        answer = answer.replace("<answer>", "").replace("</answer>", "").strip()
        print(answer)

Amazon SageMaker is a fully-managed machine learning service provided by AWS that enables developers and data scientists to build, train, and deploy machine learning models at any scale. It simplifies the machine learning process by removing the complexities and barriers that typically slow down developers. Here are the key features and benefits of Amazon SageMaker:

1. **Ease of Use**: SageMaker provides a user-friendly interface and pre-built algorithms, making it accessible even for those without deep machine learning expertise.
2. **Scalability**: It allows you to scale your machine learning workflows from a single developer to an entire team or enterprise.
3. **Integrated Development Environment (IDE)**: SageMaker Studio offers a Jupyter notebook-based IDE where you can prepare data, develop models, and deploy them.
4. **Automated Model Tuning**: SageMaker provides automatic model tuning (also known as hyperparameter optimization) to help you find the best version of your model.
5

In [174]:
response = bedrock_agent_runtime.invoke_flow(
    flowIdentifier = flowId,
    flowAliasIdentifier = flowAliasId,
    inputs = [
        { 
            "content": { 
                "document": "I have no idea what Amazon SNS is for. Could you tell me?"
            },
            "nodeName": "Start",
            "nodeOutputName": "document"
        }
    ]
)

# Process the event stream
for event in response["responseStream"]:
    if "flowOutputEvent" in event:
        # Extract just the answer content
        answer = event["flowOutputEvent"]["content"]["document"]
        # Remove the <answer> tags if present
        answer = answer.replace("<answer>", "").replace("</answer>", "").strip()
        print(answer)

### What is Amazon Simple Notification Service (Amazon SNS)?

Amazon Simple Notification Service (Amazon SNS) is a highly available, durable, secure, and fully managed pub/sub messaging service provided by AWS. It enables you to decouple microservices, distributed systems, and serverless applications by providing a flexible messaging infrastructure.

#### Key Features of Amazon SNS:

1. **Pub/Sub Messaging**:
   - **Topics**: SNS uses topics as a communication channel for sending messages to multiple subscribers. A topic acts as a message hub where publishers can send messages, and subscribers can receive them.
   - **Subscriptions**: Subscribers can be Amazon SQS queues, AWS Lambda functions, HTTP/S endpoints, email addresses, mobile push notifications, and more.

2. **High Throughput and Scalability**:
   - SNS is designed to handle high-throughput, many-to-many messaging, making it suitable for applications that need to send notifications to a large number of recipients.

3. **Decou